In [ ]:
!pip uninstall torch torchcam -y -q

In [ ]:
!pip install torch==2.1.0 torchcam==0.3.0 numpy pandas matplotlib earthengine-api geopandas -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you have torch 2.1.0 which is incompatible.
torchvision 0.21.0+cu124 requires torch==2.6.0, but you have torch 2.1.0 which is incompatible.


In [ ]:
import ee
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
ee.Authenticate()
ee.Initialize(project='ndvi-imagery')

In [ ]:
class SensorDataGenerator:
    def __init__(self, seed=42):
        self.rng = np.random.default_rng(seed)

    def generate(self, label):
        if label == "healthy":
            return [self.rng.uniform(30, 40), self.rng.uniform(18, 28),
                    self.rng.uniform(60, 80), self.rng.uniform(6.0, 7.0)]
        elif label == "unhealthy":
            return [self.rng.uniform(20, 30), self.rng.uniform(28, 35),
                    self.rng.uniform(50, 70), self.rng.uniform(5.5, 6.0)]
        else:  # drought
            return [self.rng.uniform(10, 20), self.rng.uniform(32, 40),
                    self.rng.uniform(40, 60), self.rng.uniform(5.0, 5.5)]

In [ ]:
# NDVI Download from GEE
def get_ndvi_image(coords, date_start, date_end, scale=64):
    region = ee.Geometry.Polygon(coords)
    s2 = ee.ImageCollection('COPERNICUS/S2_SR') \
           .filterBounds(region) \
           .filterDate(date_start, date_end) \
           .median()
    ndvi = s2.normalizedDifference(['B8', 'B4']).rename('NDVI')
    # Consider if sampleRectangle is the best approach for getting a 64x64 array
    # depending on the size of 'region'. If region is large, sampleRectangle might
    # sample points within the region rather than providing a structured grid.
    # For a fixed 64x64 image, you might need to use .getRegion() and reshape or
    # .reduceRegion() with a grid. However, for this error fix, we'll keep the original logic.
    arr = ndvi.sampleRectangle(region=region).get('NDVI').getInfo()
    return np.array(arr).astype(np.float32)  # Should be 64x64 if region is appropriate size

In [ ]:
# Dataset Class
class CropDataset(Dataset):
    def __init__(self, num_samples=1000):
        self.sensor_gen = SensorDataGenerator()
        self.labels = ["healthy"]*600 + ["unhealthy"]*300 + ["drought-affected"]*100
        self.ndvi_samples = []

        # Simulate NDVI images (in practice, use GEE)
        for label in self.labels:
            if label == "healthy":
                self.ndvi_samples.append(np.random.uniform(0.6, 0.9, (64, 64)))
            elif label == "unhealthy":
                self.ndvi_samples.append(np.random.uniform(0.3, 0.6, (64, 64)))
            else: # drought-affected
                self.ndvi_samples.append(np.random.uniform(0.0, 0.3, (64, 64)))

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        ndvi = self.ndvi_samples[idx]
        label = self.labels[idx]
        sensors = self.sensor_gen.generate(label)

        # Ensure labels are correctly mapped to indices
        label_map = {"healthy": 0, "unhealthy": 1, "drought-affected": 2}
        label_idx = label_map[label]

        return (
            torch.tensor(ndvi, dtype=torch.float32).unsqueeze(0),  # Add channel dim [1, 64, 64] and explicitly set dtype
            torch.tensor(sensors, dtype=torch.float32), # Ensure sensor data is float
            torch.tensor(label_idx)
        )

In [ ]:
# Model Architecture
class FusionLiteCNN(nn.Module):
    def __init__(self):
        super().__init__()
        # Input image size is 64x64
        # Conv1: (1, 64, 64) -> (32, 64, 64) after padding
        # MaxPool1: (32, 64, 64) -> (32, 32, 32)
        # Conv2: (32, 32, 32) -> (64, 32, 32) after padding
        # MaxPool2: (64, 32, 32) -> (64, 16, 16)
        # Flatten: (64, 16, 16) -> 64 * 16 * 16 = 16384 features
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten()
        )
        # MLP for sensor data (4 features)
        self.mlp = nn.Sequential(
            nn.Linear(4, 16), nn.ReLU(),
            nn.Linear(16, 32) # Outputting 32 features
        )
        # Classifier combining CNN output (16384) and MLP output (32)
        # Total features = 16384 + 32 = 16416
        self.classifier = nn.Linear(64*16*16 + 32, 3) # Outputting 3 classes

    def forward(self, img, sensors):
        img_feat = self.cnn(img)
        sensor_feat = self.mlp(sensors)
        return self.classifier(torch.cat([img_feat, sensor_feat], dim=1))


In [ ]:
# Training Loop
def train():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = FusionLiteCNN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    # num_samples=1000 is default, explicitly setting here for clarity
    dataset = CropDataset(num_samples=1000)
    loader = DataLoader(dataset, batch_size=32, shuffle=True)

    for epoch in range(10):
        running_loss = 0.0
        for img, sensors, label in loader:
            img, sensors, label = img.to(device), sensors.to(device), label.to(device)
            optimizer.zero_grad()
            outputs = model(img, sensors)
            loss = criterion(outputs, label)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * img.size(0) # Accumulate loss weighted by batch size
        epoch_loss = running_loss / len(dataset) # Calculate average loss per epoch
        print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

    torch.save(model.state_dict(), 'fusion_model.pth')
    print("Model saved!")

train()

Epoch 1, Loss: 0.2165
Epoch 2, Loss: 0.0000
Epoch 3, Loss: 0.0000
Epoch 4, Loss: 0.0000
Epoch 5, Loss: 0.0000
Epoch 6, Loss: 0.0000
Epoch 7, Loss: 0.0000
Epoch 8, Loss: 0.0000
Epoch 9, Loss: 0.0000
Epoch 10, Loss: 0.0000
Model saved!


In [ ]:
# Export to Google Drive
from google.colab import drive
drive.mount('/content/drive')
!cp fusion_model.pth /content/drive/MyDrive/

Mounted at /content/drive
